In [328]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [329]:
def read_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img)
    img = tf.image.rgb_to_grayscale(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

img = read_img("./data/dogs-vs-cats/train/cat.10000.jpg")

In [330]:
def gaussian_blur(img, kernel_size=11, sigma=5):
    def gaussion_filter(channels):
        kernel_1D = tf.range(-kernel_size // 2 + 1.0, (kernel_size // 2) + 1.0)
        xx, yy = tf.meshgrid(kernel_1D, kernel_1D)
        kernel = tf.exp(-(xx ** 2 + yy ** 2) / (2.0 * sigma ** 2))
        kernel = kernel / tf.reduce_sum(kernel)
        kernel = tf.tile(kernel[..., tf.newaxis], [1, 1, channels])
        return kernel
    
    gaussian_kernel = gaussion_filter(tf.shape(img)[-1])
    return gaussian_kernel[..., tf.newaxis]

In [448]:
kernel = gaussian_blur(img, 3, 9)

In [449]:
image = tf.reshape(img, [1, img.shape[0], img.shape[1], 1], name='image')
conv = tf.nn.conv2d(image, kernel, [1, 1, 1, 1], "SAME")

In [450]:
fig, ax = plt.subplots(1, 3, figsize=(16, 16))
ax[0].imshow(tf.reshape(kernel, (kernel.shape[0], kernel.shape[1])), interpolation='none', cmap='gray')
ax[1].imshow(tf.reshape(img, (img.shape[0], img.shape[1])), interpolation='none', cmap='gray')
ax[2].imshow(tf.reshape(conv[0], (conv.shape[1], conv.shape[2])), interpolation='none', cmap='gray')

In [451]:
def convolution(image, kernel, verbose = True, average = False):
    image_row, image_col = image.shape
    kernel_row, kernel_col, _, _ = kernel.shape
    
    output = np.zeros((image_row, image_col))
    
    pad_height = (kernel_row - 1) // 2
    pad_width = (kernel_col - 1) // 2
    
    padded_image = np.zeros((image_row + (2 * pad_height), image_col + (2 * pad_width)))
    
    padded_image[pad_height: padded_image.shape[0] - pad_height, pad_width: padded_image.shape[1] - pad_width] = image
    
    for row in range(image_row):
        for col in range(image_col):
            output[row, col] = np.sum(kernel * padded_image[row:row + kernel_row, col:col + kernel_col])
            if average:
                output[row, col] /= kernel.shape[0] * kernel.shape[1]
                
    if verbose:
        fig, ax = plt.subplots(1, 3, figsize=(16, 16))
        ax[0].imshow(image, cmap='gray')
        ax[0].set_title("Given Image")
        ax[1].imshow(padded_image, cmap='gray')
        ax[1].set_title("Padded Image")
        ax[2].imshow(output, cmap='gray')
        ax[2].set_title("Gaussian Image")
        plt.show()
                
    return output

array([[ 85.11111111, 102.88888211,  46.22222222],
       [127.66666667, 154.33333333,  69.33333333],
       [ 85.11111111, 102.88888889,  46.22222222]])

In [452]:
sample_image = np.array([[255,128,80], [255,128,80], [255,128,80]])
convolution(sample_image, kernel, average=True)